# Building OceanParcels Fields from SalishSeaCast Currents and Winds

Riffing on Raisha's 
[https://github.com/SS-Atlantis/analysis-raisha/blob/main/notebooks/contaminant-dispersal/SSAM-parcels.ipynb](https://github.com/SS-Atlantis/analysis-raisha/blob/main/notebooks/contaminant-dispersal/SSAM-parcels.ipynb)
notebook but using `pathlib` and `pprint` from Python stdlib.

* `pathlib` docs: https://docs.python.org/3/library/pathlib.html
* `pprint` docs: https://docs.python.org/3/library/pprint.html

In [5]:
import os
from pathlib import Path
from pprint import pprint

from parcels import Field, FieldSet

In [6]:
# Data Paths
currents = Path('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/')
winds = Path('/ocean/rlovindeer/Atlantis/Physics/Wind/')
sea_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSnBathymetryV17-02_a29d_efc9_4047.nc')
air_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSaAtmosphereGridV1_0f03_6268_df4b.nc')

`Path` objects are semantically distinct from strings,
and they have a bunch of useful methods and properties.

One nice one is that you can join path segments of path and files using the `/` operator:

```python
currents / '2018-01-01_URaw_variables.nc'
```
    
Here the `glob()` method is handy for building a collection of file paths by way of a wildcard pattern.
This replaces the `util.parse_wildcards()` function from `ssam_oceanparcels/Parcels_Utils/particle_tracking/parcels/`,
albeit without the file existence checks that `parse_wildcards()` does.
You could add those if you wanted with something like:

```python
assert all(p.exists() for p in u_current)
```

In [7]:
# Salish Sea NEMO Model Grid, Geo-location and Bathymetry, v17-02

# Currents
u_current = sorted([p for p in currents.glob('2018-01*URaw_variables.nc')])
v_current = sorted([p for p in currents.glob('2018-01*VRaw_variables.nc')])

filenames = {
    'U': {
        'lon': sea_grid,
        'lat': sea_grid,
        'data': u_current
    },
    'V': {
        'lon': sea_grid,
        'lat': sea_grid,
        'data': v_current
    }
}
pprint(filenames)

{'U': {'data': [PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-01_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-05_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-09_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-13_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-17_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-21_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-25_URaw_variables.nc'),
                PosixPath('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/2018-01-29_URaw_variables.nc')],
       'lat': PosixPath('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSnBathymetryV17-02_a29d_e

`pprint()` (aka pretty-print) provides nicely indented display of things like nested dictionaries.
Jupyter does some level of pretty-printing by itself,
but needs an assist when data structure start to get more complicated.

In [4]:
variables = {"U": "uVelocity", "V": "vVelocity"}
dimensions = {"lon": "longitude", "lat": "latitude", "time": "time"}
print("creating from_nemo")
fieldset = FieldSet.from_nemo(
    filenames, variables, dimensions, allow_time_extrapolation=True
)
print("creating from_nemo done")

fieldset.add_constant("decay", 1.0 * 3600.0)
print("add_constant decay")


creating from_nemo
creating from_nemo done
add_constant decay


`os.fspath()` transforms a `Path` object into a string with the correct path separators,
etc. for whatever operating system you are on.
We need to use it here because I discovered that `Field.from_netcdf()` does some checking

In [5]:
# HRDPS, Salish Sea, Atmospheric Forcing Grid, Geo-location, v1"

wind_paths = sorted([p for p in winds.glob('*_Wind_variables.nc')])
wind_filenames = {'lon': os.fspath(air_grid), 'lat': os.fspath(air_grid), 'data': wind_paths}
wind_dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}

pprint(wind_filenames)
pprint(wind_dimensions)

{'data': [PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-01_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-03_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-05_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-07_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-09_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-11_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-13_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-15_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-17_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-19_Wind_variables.nc'),
          PosixPath('/ocean/rlovindeer/Atlantis/Physics/Wind/2018-01-2

In [6]:
Uwind_field = Field.from_netcdf(wind_filenames, ('U_wind', 'u_wind'),
                                     wind_dimensions,
                                     fieldtype='U',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)
Vwind_field = Field.from_netcdf(wind_filenames, ('V_wind', 'v_wind'),
                                     wind_dimensions,
                                     fieldtype='V',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)

print('wind data loaded')

wind data loaded
